**<font color = black size=6>实验十一：支持向量机</font>**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

**<font color = blue size=4>第一部分:函数介绍</font>**

二次规划问题是形式如下的一类最优化问题：
$$
\begin{align}
\min_x \quad  &\frac{1}{2}x^TPx+q^Tx \\
s.t. \quad  &Gx\leq h \\
      &Ax=b
\end{align}
$$
对于这一类问题可以使用[cvxopt](https://cvxopt.org/userguide/coneprog.html#quadratic-programming)库的solvers.qp()函数进行求解。

以下是一个例子（参考[Solving a quadratic program](https://cvxopt.org/examples/tutorial/qp.html)）:
$$
\begin{align}
\min_x \quad  &2x_1^2+x_2^2+x_1x_2+x_1+x_2 \\
s.t. \quad  &x_1\geq 0 \\
      &x_2\geq 0 \\
      &x_1+x_2=1
\end{align}
$$
为了使用solvers.qp()函数，我们需要知道在该二次规划问题中的$P,q,G,h,A,b$矩阵分别是什么。
在该优化问题中，

* $P:=\begin{bmatrix}
    4 & 1 \\ 1 & 2
   \end{bmatrix}$,
* $q:=\begin{bmatrix}
    1 \\ 1
   \end{bmatrix}$,
* $G:=\begin{bmatrix}
    -1 & 0 \\ 0 & -1
   \end{bmatrix}$,
* $h:=\begin{bmatrix}
    0 \\ 0
   \end{bmatrix}$,
* $A:=\begin{bmatrix}
    1 & 1
   \end{bmatrix}$,
* $b:=\begin{bmatrix}
    1
   \end{bmatrix}$,
   
把这些参数送入solvers.qp()函数中即可求出解。

In [ ]:
# Tips1: cvxopt库中的matrix只接受double类型的数据
# Tips2: matrix使用列表作为参数创建矩阵和numpy.array使用列表作为参数创建矩阵是不同的
# print(matrix([[1.0, 1.0]]))
# print(np.array([[1.0, 1.0]]))
# print(matrix(np.array([[1.0, 1.0]])))
Q = 2*matrix([ [2, .5], [.5, 1] ])
p = matrix([1.0,1.0])
G = matrix([[-1.0,0.0],[0.0,-1.0]])
h = matrix([0.0,0.0])
A = matrix([1.0, 1.0], (1,2))
b = matrix(1.0)
sol=solvers.qp(Q, p, G, h, A, b)
print(sol['x'])

**<font color = blue size=4>第二部分:实验任务</font>**

1.线性可分支持向量机与硬间隔最大化

<span style="color:purple">1)  
这一部分使用的数据集'dataset1.csv'是一个线性可分的数据集。每个数据样本包含两个特征$x_1$, $x_2$以及一个标签$y\in\{1,-1\}$。  
首先，请读入数据集'dataset1.csv',把数据类型都转换成np.double类型，并画出数据集的散点图，给正样本（y为+1）和负样本（y为-1）分别标上不同的颜色。</span>

In [ ]:
# ---- Your code here ----





<span style="color:purple">2)  
求对偶问题的最优解$\lambda^*$  
在数据线性可分的场景中，为了找到一个能最好地划分正样本和负样本的超平面$\pmb{\omega}^T \pmb{x}+b=0$，我们需要求解下面这个主问题。
\begin{align}
\min_{\pmb{\omega},b}\quad &\frac12 ||\pmb{\omega}||^2\\
s.t.\quad &y_i(\pmb{\omega}^T \pmb{x}_i+b)\ge 1,i=1,...,m
\end{align}
</span>

<span style="color:purple">对应地，即求解如下对偶问题（参考课件）：</span>
$$
\begin{align}
\min_\lambda \quad  &\frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m\lambda_i\lambda_jy_iy_j\pmb{x}_i^T\pmb{x}_j-\sum_{i=1}^m\lambda_i \\
s.t. \quad  &\sum_{i=1}^m\lambda_iy_i=0 \\
      &\pmb{\lambda}\geq \pmb{0}
\end{align}
$$

这个优化问题是一个二次规划问题，可以写成如下形式：
$$
\begin{align}
\min_{\pmb{\lambda}} \quad  &\frac{1}{2}\lambda^TP\lambda+q^T\lambda \\
s.t. \quad  &G\lambda\leq h \\
      &A\lambda=b
\end{align}
$$

* $P是一个m\times m的矩阵，其中P_{ij}=y_iy_j\pmb{x}_i^T\pmb{x}_j$,
* $q是一个m\times 1的所有值都为-1的列向量，即q:=\begin{bmatrix}
    -1 & -1 & \cdots & -1
   \end{bmatrix}^T$,
* $G:=\begin{bmatrix}
    -1 & 0 & \cdots & 0 \\
    0 & -1 & \cdots & 0 \\
   \vdots & \vdots & \ddots &0 \\
   0 & 0 & 0 & -1
   \end{bmatrix}_{m\times m}=-\pmb{I},\pmb{I}为单位矩阵,$
* $h是一个m\times 1的零向量,即h:=\begin{bmatrix}
    0 & 0 & \cdots & 0
   \end{bmatrix}^T$,
* $A:=\begin{bmatrix}
    y_1 & y_2 & \cdots & y_m
   \end{bmatrix}^T$,
* $b:=\begin{bmatrix}
    0
   \end{bmatrix},一个标量$
   
把上述参数送入求解器solvers.qp()中即可得到最优解$\lambda^*$。 
 
附：$P$矩阵的一个计算方法：
设$X=\begin{bmatrix}
    x_{11} & x_{12} \\
    x_{21} & x_{22} \\
    \vdots & \vdots \\
    x_{m1} & x_{m2}
   \end{bmatrix}$,
   $Y=\begin{bmatrix}
    y_{1} \\
    y_{2} \\
    \vdots \\
    y_{m}
   \end{bmatrix}$,
   
计算$X'=\begin{bmatrix}
    x_{11}y_1 & x_{12}y_1 \\
    x_{21}y_2 & x_{22}y_2 \\
    \vdots & \vdots \\
    x_{m1}y_m & x_{m2}y_m
   \end{bmatrix}=X*Y(注意这里是星乘)$
   
则$P=X'X'^T$。

In [ ]:
# ---- Your code here ----
#如果求解报错可以尝试在solvers.qp()中添加参数kktsolver='ldl'





<span style="color:purple">3)  
求出$\pmb{\omega}^*=\sum_{i=1}^m\lambda_i^*y_i\pmb{x}_i$和$b^*=y_j-\pmb{\omega}^{*T}\pmb{x_j}$, 其中$j$为$\lambda^*$中的一个正分量$\lambda_j^*>0$的下标。  
注意：由于求解器求出来的是一个近似解，所以$\lambda^*$中很多实际上为0的分量会略大于0，这时候可以设置一个阈值把非常靠近0的那些分量筛去，再从剩下的分量中选取一个正分量来计算$b^*$,或者也可以直接取$\lambda^*$中最大的分量来计算$b^*$。</span>

In [ ]:
# ---- Your code here ----





<span style="color:purple">4)  
画出数据集的散点图，给正样本（y为+1）和负样本（y为-1）分别标上不同的颜色，再为支持向量（训练数据中$\lambda_j^*>0$的对应的样本）标上不同的颜色，并画出决策边界$\pmb{\omega}^{*T}\pmb{x}+b=0$和间隔边界$\pmb{\omega}^{*T}\pmb{x}+b=1$与$\pmb{\omega}^{*T}\pmb{x}+b=-1$。</span>

In [ ]:
# ---- Your code here ----






2.线性支持向量机与软间隔最大化

<span style="color:purple">1)  
这一部分使用的数据集'dataset2.csv'是一个数据近似线性可分的数据集。每个数据样本同样包含两个特征$x_1$, $x_2$以及一个标签$y\in\{1,-1\}$。   
读入数据集'dataset2.csv',把数据类型都转换成np.double类型，并画出数据集的散点图，给正样本（y为+1）和负样本（y为-1）分别标上不同的颜色。</span>

In [ ]:
# ---- Your code here ----





<span style="color:purple">2)  
求对偶问题的最优解$\lambda^*$  
在数据近似线性可分的场景中，为了找到一个能最好地划分正样本和负样本的超平面$\pmb{\omega}^T \pmb{x}+b=0$，我们需要求解下面这个主问题。
\begin{align}
\min_{\pmb{\omega},b,\xi_i}\quad &\frac12 ||\pmb{\omega}||^2+C\times\sum_{i=1}^m \xi_i\\
s.t.\quad &y_i(\pmb{\omega}^T \pmb{x}_i+b)\ge 1-\xi_i,i=1,...,m\\
&\xi_i\ge 0, i=1,...,m\\
\end{align}
</span>

<span style="color:purple">对应地，我们需要选择一个参数C，求解如下对偶问题（参考课件）：</span>
$$
\begin{align}
\min_\lambda \quad  &\frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m\lambda_i\lambda_jy_iy_j\pmb{x}_i^T\pmb{x}_j-\sum_{i=1}^m\lambda_i \\
s.t. \quad  &\sum_{i=1}^m\lambda_iy_i=0 \\
      &\pmb{0}\leq \pmb{\lambda}\leq C 
\end{align}
$$

同样地，这个问题也可以写成如下形式：  
$$
\begin{align}
\min_{\lambda} \quad  &\frac{1}{2}\lambda^TP\lambda+q^T\lambda \\
s.t. \quad  &G\lambda\leq h \\
      &A\lambda=b
\end{align}
$$


* $G:=\begin{bmatrix}
    -1 & 0 & \cdots & 0 \\
    0 & -1 & \cdots & 0 \\
   \vdots & \vdots & \ddots &0 \\
   0 & 0 & 0 & -1 \\
   1 & 0 & \cdots & 0 \\
    0 & 1 & \cdots & 0 \\
   \vdots & \vdots & \ddots &0 \\
   0 & 0 & 0 & 1
   \end{bmatrix}_{2m\times m}=\begin{bmatrix}
    -\pmb{I} \\
    \pmb{I}
   \end{bmatrix},\pmb{I}为单位矩阵,$
* $h:=\begin{bmatrix}
    0 \\
    0 \\
    \vdots \\
    0 \\
    C \\
    C \\
    \vdots \\
    C
   \end{bmatrix}_{2m\times 1}, 即一个m\times 1的零列向量与一个m\times 1的分量全为C的列向量上下拼接$,
* $P,q,A,b$与硬间隔优化问题中的矩阵相同。  
* 参数$C$请自行选择。

In [ ]:
# ---- Your code here ----






<span style="color:purple">3)  
求出$\pmb{\omega}^*=\sum_{i=1}^m\lambda_i^*y_i\pmb{x}_i$和$b^*=y_j-\pmb{\omega}^{*T}\pmb{x_j}$, 其中$j$为$\lambda^*$中的一个正分量$0<\lambda_j^*<C$的下标。与硬间隔优化问题同理，应该避免选择非常接近0和非常接近$C$的分量。</span>

In [ ]:
# ---- Your code here ----






<span style="color:purple">4)  
画出数据集的散点图，给正样本（y为+1）和负样本（y为-1）分别标上不同的颜色，再为支持向量（训练数据中$\lambda_j^*>0$的对应的样本）标上不同的颜色，并画出决策边界$\pmb{\omega}^{*T}\pmb{x}+b=0$和间隔边界$\pmb{\omega}^{*T}\pmb{x}+b=1$与$\pmb{\omega}^{*T}\pmb{x}+b=-1$。</span>

In [ ]:
# ---- Your code here ----







3.非线性支持向量机与核函数

[Raisin Dataset](https://www.kaggle.com/datasets/muratkokludataset/raisin-dataset)是一个葡萄干的数据集，总共有900个样本，每个样本包含7个(都是连续的)特征以及1个标签，每个标签只有两种可能取值。本次实验已经按照8：2的比例划分成了训练数据集'Raisin_train.csv'以及测试数据集'Raisin_test.csv'，且每个数据集都已经做了特征归一化处理以及把标签的值替换成了+1和-1。

<span style="color:purple">1) 读入训练数据集'Raisin_train.csv',把数据类型都转换成np.double类型。</span>

In [ ]:
# ---- Your code here ----






<span style="color:purple">2)  
求对偶问题的最优解$\lambda^*$  
在数据非线性可分的场景中，我们需要求解下面这个主问题。
\begin{align}
\min_{\pmb{\omega},b,\xi_i}\quad &\frac12 ||\pmb{\omega}||^2+C\times\sum_{i=1}^m \xi_i\\
s.t.\quad &y_i(\pmb{\omega}^T \phi(\pmb{x}_i)+b)\ge 1-\xi_i,i=1,...,m\\
&\xi_i\ge 0, i=1,...,m\\
\end{align}
</span>

<span style="color:purple">对应地，我们需要：  
选择一个核函数$K(\pmb{x},\pmb{z})$以及参数C，求解如下对偶问题（参考课件）：</span>
$$
\begin{align}
\min_\lambda\quad   &\frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m\lambda_i\lambda_jy_iy_jK(\pmb{x}_i,\pmb{x}_j)-\sum_{i=1}^m\lambda_i \\
s.t. \quad  &\sum_{i=1}^m\lambda_iy_i=0 \\
      &0\leq \lambda_i \leq C, i=1,...,m 
\end{align}
$$

相较于硬间隔最大化的优化问题，该优化问题仅需要对矩阵$P$做改动。
从以下常用的核函数中选择一个作为该优化问题中的$K$（其中的参数请自行进行调整）：
* 线性核：$K(\pmb{x},\pmb{z})=\pmb{x}^T\pmb{z}$
* 多项式核：$K(\pmb{x},\pmb{z})=(\pmb{x}^T\pmb{z}+1)^p$
* 高斯核：$K(\pmb{x},\pmb{z})=exp(-\frac{\parallel \pmb{x}-\pmb{z} \parallel^2}{2\sigma^2})$
* 拉普拉斯核：$K(\pmb{x},\pmb{z})=exp(-\frac{\parallel \pmb{x}-\pmb{z} \parallel}{\sigma})$
* Sigmoid核：$K(\pmb{x},\pmb{z})=tanh(\beta\pmb{x}^T\pmb{z}+\theta)$

则$P是一个m\times m的矩阵，其中P_{ij}=y_iy_jK(\pmb{x_i},\pmb{x_j})$。

In [ ]:
# ---- Your code here ----






<span style="color:purple">3)  
求出$b^*=y_j-\sum_{i=1}^m \lambda_i^*y_iK(\pmb{x_i},\pmb{x_j})$, 其中$j$为$\lambda^*$中的一个正分量$0<\lambda_j^*<C$的下标。</span>

In [ ]:
# ---- Your code here ----






<span style="color:purple">4) 读入测试数据集'Raisin_test.csv',用分类决策函数$f(\pmb{x})=sign(\sum_{i=1}^m \lambda_i^*y_iK(\pmb{x}_i,\pmb{x})+b^*)$（注意这里的$m,\lambda_i^*,y_i,\pmb{x}_i$是训练集的, $\pmb{x}$是测试集的）进行预测，输出预测准确率。</span>

In [ ]:
# ---- Your code here ----








**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告下周五实验课(12月1号 14:20)上课前提交报告  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验十一(支持向量机)的实验报告上交地址:https://send2me.cn/TRRgKD4K/RX29mant_U152w

三、课堂课件获取地址:https://www.jianguoyun.com/p/DZKTh-IQp5WhChiIn6gFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DWOjj7kQp5WhChi0nqkFIAA